In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert import focuss, inverse_msp
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

In [74]:
fwd.values()["data"].shape

TypeError: 'dict_values' object is not subscriptable

# Get Forward Model

In [2]:
info = get_info(kind='biosemi128')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.5s remaining:    2.6s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.5s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


# Get sample data

In [58]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=4, extents=(1, 40), duration_of_trial=1, target_snr=99999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
M = evoked.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 250.19it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00,  4.52it/s]

Using control points [0.00000000e+00 0.00000000e+00 3.12518578e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Minimum Norm Estimate

In [ ]:
alpha = 0.001
# if n_chans>n_dipoles:
# D_MNE = np.linalg.inv(leadfield.T @ leadfield + alpha * np.identity(n_dipoles)) @ leadfield.T @ M
# else:
D_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans)) @ M


stc_hat = stc.copy()
stc_hat.data = D_MNE
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'MNE', 'title',
               font_size=14)

# Weighted Minimum Norm Estimate

In [4]:
alpha = 0.001
omega = np.diag(np.linalg.norm(leadfield, axis=0))
I_3 = np.identity(3)
W = omega # np.kron(omega, I_3)

D_WMNE = np.linalg.inv(W.T @ W) @ leadfield.T  @ np.linalg.inv(leadfield @ np.linalg.inv(W.T @ W) @ leadfield.T + alpha * np.identity(n_chans)) @ M

stc_hat = stc.copy()
stc_hat.data = D_WMNE
r = np.median([pearsonr(abs(s_pred), abs(s_true))[0] for s_pred, s_true in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'wMNE (r={r:.2f})', 'title',
               font_size=14)

Using control points [1.73250790e-09 2.43647303e-09 1.20457740e-08]


# MNE with FOCUSS (Focal underdetermined system solution)

In [ ]:
alpha = 0.001

D_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans)) @ M
D_FOCUSS = focuss(D_MNE, M, leadfield, alpha)



stc_hat = stc.copy()
stc_hat.data = D_FOCUSS
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'MNE FOCUSS', 'title',
            font_size=14)

# Contextual Minimum Norm

In [9]:
from invert import inverse_loreta, contextualize, contextualize_bd
import tensorflow as tf
from scipy.stats import pearsonr


stc_instant = inverse_loreta(M, leadfield, fwd)
D_cmne = contextualize_bd(stc_instant, leadfield, num_epochs=20)

stc = sim.source_data[0]


stc_hat = stc.copy()
stc_hat.data = D_cmne
r = np.median([pearsonr(abs(s_pred), abs(s_true))[0] for s_pred, s_true in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'CMNE (r={r:.2f})', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               723456    
_________________________________________________________________
dense_1 (Dense)              (None, 1284)              165636    
Total params: 889,092
Trainable params: 889,092
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
49/49 [==============================] - 13s 209ms/step - loss: 0.0106 - cosine_similarity: -0.4553 - val_loss: 0.0111 - val_cosine_similarity: -0.5078
Epoch 2/20
49/49 [==============================] - 10s 198ms/step - loss: 0.0060 - cosine_similarity: -0.7559 - val_loss: 0.0112 - val_cosine_similarity: -0.5283
Epoch 3/20
49/49 [==============================] - 10s 200ms/step - loss: 0.0061 - cosine_similarity: -0.7723 - val_loss: 0.0107 - val_cosine_sim

# dSPM

In [ ]:
alpha = 0.001

noise_cov = np.identity(n_chans) + np.random.rand(n_chans, n_chans)*0.1
source_cov = np.identity(n_dipoles)

M_norm = (1/np.sqrt(noise_cov)) @ M
G_norm = (1/np.sqrt(noise_cov)) @ leadfield

K = source_cov @ G_norm.T @ np.linalg.inv(G_norm @ source_cov @ G_norm.T + alpha**2 * np.identity(n_chans))
W_dSPM = np.diag(np.sqrt(1/np.diagonal(K @ noise_cov @ K.T)))
K_dSPM = W_dSPM @ K
D_dSPM = K_dSPM @ M_norm


stc_hat = stc.copy()
stc_hat.data = D_dSPM
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'dSPM', 'title',
               font_size=14)

# LORETA

In [55]:
alpha = 2
adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
B = np.diag(np.linalg.norm(leadfield, axis=0))
laplace_operator = laplacian(adjacency)
D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ M
# D_LOR = np.linalg.inv(B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ (leadfield @ np.linalg.pinv(B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T + alpha * np.identity(n_chans) ) @ M

stc_hat = stc.copy()
D_LOR /= np.abs(D_LOR).max()
stc_hat.data = D_LOR
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'LORETA (r={r:.2f})', 'title',
            font_size=14)

-- number of adjacent vertices : 1284
Using control points [0.1601314  0.18994773 0.4961905 ]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [0.27330654 0.32540344 0.49861047]
Using control points [0.27330654 0.32540344 0.49861047]
Using control points [0.00000000e+00 3.61118105e-08 3.61118105e-08]


# sLORETA

In [75]:
alpha = 0.001
K_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans))
W_diag = 1 / np.diag(K_MNE @ leadfield)

W_slor = np.diag(W_diag)

W_slor = np.sqrt(W_slor)

K_slor = W_slor @ K_MNE
D_SLOR = K_slor @ M
D_SLOR[np.isnan(D_SLOR)] = 0

stc_hat = stc.copy()
stc_hat.data = D_SLOR
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'sLORETA (r={r:.2f})', 'title',
            font_size=14)

Using control points [3.13785569e-09 3.88109967e-09 1.48665829e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# eLORETA

In [62]:
from invert import calc_eloreta_D
stop_crit = 0.005
alpha = 0.001

D, C = calc_eloreta_D(leadfield, alpha, stop_crit=stop_crit)
K_elor = np.linalg.inv(D) @ leadfield.T @ np.linalg.inv( leadfield @ np.linalg.inv(D) @ leadfield.T + alpha * np.identity(n_chans) )
D_ELOR = K_elor @ M

stc_hat = stc.copy()
stc_hat.data = D_ELOR
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'eLORETA (r={r:.2f})', 'title',
            font_size=14)

Optimizing eLORETA weight matrix W...
	rep 1
averagePercentChange=74.36 %
	rep 2
averagePercentChange=44.29 %
	rep 3
averagePercentChange=23.47 %
	rep 4
averagePercentChange=11.93 %
	rep 5
averagePercentChange=5.98 %
	rep 6
averagePercentChange=2.98 %
	rep 7
averagePercentChange=1.48 %
	rep 8
averagePercentChange=0.74 %
	rep 9
averagePercentChange=0.37 %
	rep 10
averagePercentChange=0.18 %
	rep 11
averagePercentChange=0.09 %
	rep 12
averagePercentChange=0.05 %
	rep 13
averagePercentChange=0.02 %
	rep 14
averagePercentChange=0.01 %
	rep 15
averagePercentChange=0.01 %
	rep 16
averagePercentChange=0.00 %
	...converged...
	...done!
Using control points [8.40225605e-10 1.09794372e-09 5.35543771e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [1.38654979e-09 2.05363207e-09 5.79856281e-09]
Using control points [4.01700132e-09 5.29459277e-09 1.68636041e-08]
Using control points [1.93358500e-09 2.78080877e-09 7.91422418e-09]
Using control points [2.94625895e-10 3.52861089e-10 6.20076212e-10]
Using control points [1.33016095e-09 1.57318636e-09 2.23031258e-09]


# LAURA

In [8]:
alpha = 200
drop_off = 2
d = cdist(pos, pos)
# Get the adjacency matrix of the source spaces
adj = mne.spatial_src_adjacency(fwd["src"], verbose=0).toarray()
for i in range(d.shape[0]):
    # find dipoles that are no neighbor to dipole i
    non_neighbors = np.where(~adj.astype(bool)[i, :])[0]
    # append dipole itself
    non_neighbors = np.append(non_neighbors, i)
    # set non-neighbors to zero
    d[i, non_neighbors] = 0
A = -d**-drop_off
A[np.isinf(A)] = 0
W = np.identity(A.shape[0])
M_j = W @ A

# Source Space metric
W_j = np.linalg.inv(M_j.T @ M_j)
W_j_inv = np.linalg.inv(W_j)

W_d = np.linalg.inv(np.identity(n_chans))
noise_term = (alpha**2) * np.linalg.inv(W_d)
G = W_j_inv @ leadfield.T @ np.linalg.inv(leadfield @ W_j_inv @ leadfield.T + noise_term)
D_LAURA = G @ M

stc_hat.data = D_LAURA
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'LAURA (r={r:.2f})', 'title',
            font_size=14)

C:\Users\Lukas\AppData\Local\Temp/ipykernel_936/478972351.py:13: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


Using control points [9.67561006e-10 1.30718866e-09 4.98922947e-09]


# VARETA

In [ ]:
# 1) J & JVareta given; find A
# 2) A given, find J
# 3) repeat until convergence

In [ ]:
 
# alpha = 0.001
# adjacency = mne.spatial_src_adjacency(fwd['src'], verbose=0).toarray()
# B = np.diag(np.linalg.norm(leadfield, axis=0))
# L = laplacian(adjacency)  # non-singular univariate discrete laplacian -> is that correct?
# D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ L.T @ L @ B) @ leadfield.T @ M
# D_Last = deepcopy(D_LOR)[:, 0][:, np.newaxis]
# W = np.diag(np.linalg.norm(leadfield, axis=0))  # WMNE weight matrix (depth weighting)
# tau = 1  # controls smoothness
# alpha_2 = 1  # controls importance of grid point
# # for t in range(M.shape[1]):
# t = 0
# A = np.identity(n_dipoles)  # in paper: large lambda
    
# # for _ in range(10):
# term_1 = np.linalg.norm( M[:, t] - leadfield @ D_Last )
# term_2 = np.linalg.norm( A@L * W * D_Last )
# term_3 = tau**2 * np.linalg.norm( L * np.diag(np.log(np.diagonal(A))) - alpha_2 )


# # D_VAR = term_1 + term_2 + term_3

# from scipy.optimize import minimize
# def find_lambda(A, m, leadfield, D_Last, L, W, tau, alpha_2):
#     A = A.reshape(leadfield.shape[1], leadfield.shape[1])
#     term_1 = np.linalg.norm( M[:, t] - leadfield @ D_Last )
#     term_2 = np.linalg.norm( A@L * W * D_Last )
#     term_3 = tau**2 * np.linalg.norm( L * np.diag(np.log(np.diagonal(A))) - alpha_2 )
#     return term_1 + term_2 + term_3
# # find_lambda(A, M[:, t], leadfield, D_Last, L, W, tau, alpha_2)
# minimize(find_lambda, A.flatten(), args=(M[:, t], leadfield, D_Last, L, W, tau, alpha_2), method='L-BFGS-B')

# S-MAP

In [69]:
alpha = 0.001
adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
B = np.diag(np.linalg.norm(leadfield, axis=0))
laplace_operator = laplacian(adjacency)
D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ M

gradient = np.gradient(B)[0] #np.gradient(B)[0]
D_SMAP = np.linalg.inv(leadfield.T @ leadfield + alpha * gradient.T @ gradient) @ leadfield.T @ M
# D_SMAP = np.linalg.inv(gradient.T @ gradient) @ leadfield.T @ np.linalg.inv(leadfield @ np.linalg.inv( gradient.T @ gradient ) @ leadfield.T + alpha * np.identity(n_chans)) @ M

stc_hat = stc.copy()
stc_hat.data = D_SMAP
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'S-MAP (r={r:.2f})', 'title',
            font_size=14)

-- number of adjacent vertices : 1284
Using control points [2.42637500e-07 2.85939694e-07 7.07739138e-07]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# ESINET

In [13]:
from esinet import Net
import tensorflow as tf
sim_train = Simulation(fwd, info, settings=dict(duration_of_trial=0, target_snr=999)).simulate(10000)
net = Net(fwd, n_dense_units=512, n_lstm_layers=0, activation_function='tanh').fit(sim_train)
# del sim_train

D_ESINET = net.predict(sim.eeg_data[0])[0].data

stc_hat = stc.copy()
stc_hat.data = D_ESINET
r = np.mean([pearsonr(a, b)[0] for a, b in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'ESINET (r={r:.2f})', 'title',
            font_size=14)


D_cESINET = contextualize_bd(D_ESINET, fwd)
stc_hat.data = D_cESINET
brain = stc_hat.plot(**pp)
r = np.mean([pearsonr(a, b)[0] for a, b in zip(stc_hat.data.T, stc.data.T)])
brain.add_text(0.1, 0.9, f'cESINET (r={r:.2f})', 'title',
            font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 10000/10000 [00:00<00:00, 29888.50it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 10000/10000 [00:14<00:00, 690.33it/s]


preprocess data
Model: "Dense-model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FC_0 (TimeDistributed)       (None, None, 512)         66048     
_________________________________________________________________
Drop_0 (Dropout)             (None, None, 512)         0         
_________________________________________________________________
FC_Out (TimeDistributed)     (None, None, 1284)        658692    
Total params: 724,740
Trainable params: 724,740
Non-trainable params: 0
_________________________________________________________________
fit model
Epoch 1/50
1125/1125 [==============================] - 17s 15ms/step - loss: -0.3748 - mae: 0.2079 - val_loss: -0.4356 - val_mae: 0.2138
Epoch 2/50
1125/1125 [==============================] - 15s 13ms/step - loss: -0.4362 - mae: 0.2458 - val_loss: -0.4560 - val_mae: 0.2591
Epoch 3/50
1125/1125 [==============================] - 14s 13ms/step - loss:

c:\Users\lukas\virtualenvs\esienv\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


Using control points [3.39027658e-09 5.02316581e-09 2.02890425e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               723456    
_________________________________________________________________
dense_3 (Dense)              (None, 1284)              165636    
Total params: 889,092
Trainable params: 889,092
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
49/49 [==============================] - 18s 330ms/step - loss: 0.0168 - cosine_similarity: -0.5016 - val_loss: 0.0180 - val_cosine_similarity: -0.5434
Epoch 2/100
49/49 [==============================] - 16s 325ms

Using control points [0.10866356 0.1490573  0.48585741]


# Backus-Gilbert

In [6]:
# Calculate distance matrix
dist = cdist(pos, pos)

print("Get W_BG")
W_BG = []
for i in range(n_dipoles):
    W_gamma_BG = np.diag(dist[i, :])
    W_BG.append(W_gamma_BG)

print("Get C")
C = []
for i in range(n_dipoles):
    C_gamma = leadfield @ W_BG[i] @ leadfield.T
    C.append(C_gamma)

print("Get F")
F = leadfield @ leadfield.T

print("Get E")
E = []
for i in range(n_dipoles):
    E_gamma = C[i] + F
    E.append(E_gamma)

print("Get L")
L = leadfield @ np.ones((n_dipoles, 1))

print("Get T")
T = []
for i in range(n_dipoles):
    E_gamma_pinv = np.linalg.pinv(E[i])
    T_gamma = (E_gamma_pinv @ L) / (L.T @ E_gamma_pinv @ L)
    T.append(T_gamma)

T_final = np.stack(T, axis=0)[:, :, 0]
D_BG = T_final @ M

stc_hat = stc.copy()
stc_hat.data = D_BG
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'Backus-Gilbert', 'title',
            font_size=14)

Get W_BG
Get C
Get F
Get E
Get L
Get T
Using control points [3.08304612e-10 3.54493086e-10 7.33911871e-10]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Multiple Sparse Priors

In [7]:
D_MSP = inverse_msp(evoked, fwd)

stc_hat = stc.copy()
stc_hat.data = D_MSP
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'MSP (r={r:.2f})', 'title',
            font_size=14)

# D_bLORETA = inverse_msp(evoked, fwd, inversion_type="LORETA")

# stc_hat = stc.copy()
# stc_hat.data = D_bLORETA
# r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
# brain = stc_hat.plot(**pp)
# brain.add_text(0.1, 0.9, f'Bayesian LORETA (r={r:.2f})', 'title',
#             font_size=14)

Using 4 temporal mode(s)
dedh.shape:  (3256, 193) v[0].shape:  (68,) v[1].shape:  (15,)
Iteration 1. Free Energy Improvement: 9.32
Iteration 2. Free Energy Improvement: 5.61
Iteration 3. Free Energy Improvement: 5.87
Iteration 4. Free Energy Improvement: 4.82
Iteration 5. Free Energy Improvement: 8.22
Iteration 6. Free Energy Improvement: 11.57
Iteration 7. Free Energy Improvement: 5.91
Iteration 8. Free Energy Improvement: 3.61
Iteration 9. Free Energy Improvement: 3.22
Iteration 10. Free Energy Improvement: 3.10
Iteration 11. Free Energy Improvement: 3.37
Iteration 12. Free Energy Improvement: 3.44
Iteration 13. Free Energy Improvement: 13.46
Iteration 14. Free Energy Improvement: 2.47
Iteration 15. Free Energy Improvement: 2.69
Iteration 16. Free Energy Improvement: 3.02
Iteration 17. Free Energy Improvement: 13.24
Iteration 18. Free Energy Improvement: 2.87
Iteration 19. Free Energy Improvement: 2.65
Iteration 20. Free Energy Improvement: 2.50
Iteration 21. Free Energy Improvement:

Using control points [0.00000000e+00 0.00000000e+00 2.57463767e-08]
Using control points [4.89426729e-10 7.04771488e-10 2.68613671e-09]
Using control points [3.36576201e-10 3.62787114e-10 4.33057412e-10]
Using control points [0.00000000e+00 0.00000000e+00 2.57463767e-08]
Using control points [4.89426729e-10 7.04771488e-10 2.68613671e-09]
Using control points [5.63086327e-10 7.34752712e-10 3.99230750e-09]
Using control points [0.00000000e+00 0.00000000e+00 2.10034406e-08]
Using control points [2.82157517e-10 3.28482218e-10 7.02647755e-10]
Using control points [4.28358364e-09 5.45149920e-09 1.15753074e-08]
Using control points [0.04710826 0.06561769 0.21398079]



# LUCAS

In [ ]:
stc_hat.data = np.mean([D_MNE , D_WMNE, D_LOR, D_SLOR, D_ELOR, D_LAURA, D_SMAP, D_ESINET, D_BG, D_MSP], axis=0)
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'LUCAS', 'title',
            font_size=14)

stc_hat.data = contextualize(stc_hat.data, fwd)
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'cLUCAS', 'title',
            font_size=14)

# Saved for later

In [ ]:
from esinet.util import get_eeg_from_source
evoked_hat = get_eeg_from_source(stc_hat, fwd, info, tmin=stc.tmin)
evoked_hat.plot_topomap()

In [ ]:
evoked.plot_topomap()